In [2]:
import pandas as pd

import numpy as np

from datetime import datetime

import requests
from requests.exceptions import RequestException
import json

from datetime import datetime




IMPORT RELEVANT CSV FILE


In [43]:
# Main CDM Registry project datasource from Database PA POA
df_activities= pd.read_excel('./input/CDM-Activities-November.xlsx')

# Country and geographic identifier file for easier mapping
df_country_code = pd.read_excel('./input/cdm-country-code.xlsx')


# Countries in CADTrust, to check with the mismatch from UNFCCC in later data processing
df_cadt_country = pd.read_excel("./input/CADTrust_Country.xlsx")

# LOAD UNEP file for PDD mapping
df_unep = pd.read_excel("./input/cdm-pipeline.xlsx")







INITIAL CLEANUP


In [44]:

# FILL CDM project reference number NAN value to 0, since there are almost 3000 of them

df_activities['CDM project reference number'] = df_activities['CDM project reference number'].fillna(0).astype(int)


# Check projects with more than one host countries/ "List of host countries (ISO 2)"
more_countries = df_activities[df_activities["List of host countries (ISO 2)"].str.len() > 2]

# Create excel files for projects with more than one countries/ "List of host countries (ISO 2)",
more_countries.to_excel("./output/CDM_projects_many_countries.xlsx")

# Map to CAD Trust based on the first country in the column
df_activities["List of host countries (ISO 2)"] = df_activities["List of host countries (ISO 2)"].str[:2]

# Apply geographic identifier that is suitable for CAD Trust mapping
df_country_code['geographicIdentifier'] = df_country_code.apply(
    lambda row: "{\"latitude\":" + str(row["latitude"]) + ",\"longitude\":" + str(row["longitude"]) + "}", axis=1
)

# Key Value mapping of PDD Consultant for CAD Trust projectDeveloper
pdd_developer_map = df_unep.set_index('Unique project ID')['PDD Consultant'].to_dict()

# Key Value mapping of reference number to unique identifier that will be useful for ISSUANCES and UNITS
id_to_id = df_activities.set_index('CDM project reference number')['Unique project identifier (traceable with Google)'].to_dict()

#Key Value mapping Country Code to CAD Trust Country Field
country_name_map = df_country_code.set_index('country')['cadtcountry'].to_dict()


#Key Value mapping Country Code to Geography Map
geography_map = df_country_code.set_index('name')['geographicIdentifier'].to_dict()

#Key Value mapping verification report date
verification_report_date_map = df_activities.set_index('Unique project identifier (traceable with Google)')['Start of first crediting period'].to_dict()


# Map to status fields from CDM to CAD Trust Project Status Picklist Value
project_status_map = {
    "Registered": "Registered",
    "Rejected": "Withdrawn",
    "Withdrawn":"Withdrawn",
    "WithdrawnBeforePublication": "Withdrawn",
    "Withdrawn Before Publication": "Withdrawn",
    "Deregistered": "De-registered",
    "Pending Publication": "Authorized",
    "Provisional": "Validated",
    "Validation Replaced": "Validated",
    "Requesting Registration": "Listed",
    "Validation Public": "Validated",
    "Validation Terminated": "Withdrawn",
    "NA":"Listed",
    "":"Listed",
    np.nan: "Listed"  # Handling actual NaN values
}
# Map from sector code number to it's description
sector_code_map = {
  "1": 'Energy industries (renewable - / non-renewable sources)',
  "2": 'Energy distribution',
  "3": 'Energy demand',
  "4": 'Manufacturing industries',
  "5": 'Chemical industries',
  "6": 'Construction',
  "7": 'Transport',
  "8": 'Mining/mineral production',
  "9": 'Metal production',
  "10": 'Fugitive emissions from fuels (solid, oil and gas)',
  "11": 'Fugitive emissions from production and consumption of halocarbons and sulphur hexafluoride',
  "12": 'Solvent use',
  "13": 'Waste handling and disposal',
  "14": 'Afforestation and reforestation',
  "15": 'Agriculture'
}


# basic cleanup
def fillna_by_dtype(df):
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):  # Check if the column is numeric
            df[column] = df[column].fillna(0)
        elif pd.api.types.is_object_dtype(df[column]):  # Check if the column is of object type (e.g., strings)
            df[column] = df[column].fillna('NA')
        elif pd.api.types.is_datetime64_any_dtype(df[column]):  # Check if the column is datetime
            df[column] = df[column].fillna(pd.Timestamp.min)
    return df

    


set_cadt_country = set(df_cadt_country['Country'])
set_country_code = set(df_country_code['name'])

# Find non-matching elements
non_matching_in_cadt_country = set_cadt_country - set_country_code
non_matching_in_country_code = set_country_code - set_cadt_country




In [45]:
# Rename the df column to match CADTrust field

rename_columns  = {
  "Unique project identifier (traceable with Google)": "originProjectId",
  "Registration project title": "projectName",
  "Type of CDM project: PA/PoA": "program",
  "Sectoral scope number(s)": "sector",
  "Project type (UNEP DTU)": "projectType",
  "Website project status": "projectStatus",
  "Methodologies used at registration": "methodology",
  "DOE": "validationBody",
  "Start of validation": "validationDate",
  "Country subregion":"inCountryRegion",
  "List of host countries (ISO 2)": "country",

}

df_project = df_activities[rename_columns.keys()].rename(columns=rename_columns)

# Adding other relevant values to CAD Trust field
df_project["registryOfOrigin"]= "CDM Registry"
df_project['unitMetric'] = 'tCO2e'
df_project['methodology'] = "CDM - " + df_project['methodology']
df_project['projectLink'] = 'https://cdm.unfccc.int'
df_project['projectId'] = df_project['originProjectId']
df_project['coveredByNDC'] = "Unknown"
df_project['projectStatusDate'] = datetime.now().strftime('%Y-%m-%d')

# Map the countries
df_project['country'] = df_project['country'].map(country_name_map)
df_project['sector'] = df_project['sector'].map(sector_code_map)
df_project['projectDeveloper'] = df_project['projectId'].map(pdd_developer_map)
df_project["geographicIdentifier"]= df_project['country'].map(geography_map)
df_project['description'] = df_project['projectName']
df_project['ndcInformation'] = "NA"

# Implement the fillna function created earlier
df_project = fillna_by_dtype(df_project)
df_project["projectStatus"]= df_project['projectStatus'].map(project_status_map)

# Output to check the new DataFrame
df_project.to_excel("./output/CDM-Projects-mapped.xlsx")

# df_project['country'].head(10)





Findings After Failing to Push data to Chia staging API

In [46]:
# Yemen and Cape Verde not in CAD Trust picklist. This require approval from technical committee otherwise we cant push the data
# 5 projects in total
df_project_yemen_cape_verde = df_project[(df_project['country'] == 'Yemen') | (df_project['country'] == 'Cape Verde')]
# df_with_nan_cdmID has One nan value which is project CDM 10121 row 8459

df_with_nan_cdmID = df_project[df_project['projectId']=='NA']

# Projects that have projectID, but the rest are not filled properly
df_with_nan_validation_date = df_project[df_project['validationDate']=='NA']

# Telling this issue to unfccc
df_to_inform_UNFCCC_1 = df_activities[df_activities['Unique project identifier (traceable with Google)']
                                        .isna()]

df_to_inform_UNFCCC_2 = df_activities[df_activities['Unique project identifier (traceable with Google)']
                                        .isin(df_with_nan_validation_date['projectId'])]

# Concatenating the two DataFrame into one
df_to_inform_UNFCCC = pd.concat([df_to_inform_UNFCCC_1, df_to_inform_UNFCCC_2])

df_to_inform_UNFCCC.to_excel("./output/Uncleaned_CDM_Projects.xlsx")



DROP THE ROWS OF DATA MENTIONED ABOVE. AFTER THAT, the API call should be smoothly post all the entire data to CAD Trust chia datalayer local staging mode

In [47]:

df_cleaned_project = df_project[~((df_project['country'] == 'Yemen') | (df_project['country'] == 'Cape Verde')| (df_project['country'] == 'Bhutan'))]

# Select rows where projectId is not 'NA'
df_cleaned_project = df_cleaned_project[df_cleaned_project['projectId'] != 'NA']


# Select rows where validationDate is not 'NA'
df_cleaned_project = df_cleaned_project[df_cleaned_project['validationDate'] != 'NA']

df_cleaned_project.to_excel("./output/cdm_projects_mapped_cleaned.xlsx")
# df_cleaned_project
df_bhutan = df_project[df_project['country'] == 'Bhutan']
df_bhutan

df_cleaned_project

,originProjectId,projectName,program,sector,projectType,projectStatus,methodology,validationBody,validationDate,inCountryRegion,...,registryOfOrigin,unitMetric,projectLink,projectId,coveredByNDC,projectStatusDate,projectDeveloper,geographicIdentifier,description,ndcInformation
0,AuxID,Project for GHG emission reduction by thermal ...,PA,Fugitive emissions from production and consump...,HFCs,Registered,CDM - AM0001,SGS United Kingdom Limited (SGS),2003-12-01 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,AuxID,Unknown,2023-12-07,PricewaterhouseCoopers,"{""latitude"":20.593684,""longitude"":78.96288}",Project for GHG emission reduction by thermal ...,NA
1,LJ80OQZR2JVOLTZZX4Y0EGB1N18AVO,HFC Decomposition Project in Ulsan,PA,Fugitive emissions from production and consump...,HFCs,Registered,CDM - AM0001,Japan Quality Assurance Organisation (JQA),2003-12-11 00:00:00,Eastern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,LJ80OQZR2JVOLTZZX4Y0EGB1N18AVO,Unknown,2023-12-07,Climate Experts,NA,HFC Decomposition Project in Ulsan,NA
2,SQA16OAS75TKWYZTTKK7G1LO0O8CMU,Brazil NovaGerar Landfill Gas to Energy Project,PA,Waste handling and disposal,Landfill gas,Registered,CDM - AM0003,Det Norske Veritas- CUK,2004-04-05 00:00:00,South America,...,CDM Registry,tCO2e,https://cdm.unfccc.int,SQA16OAS75TKWYZTTKK7G1LO0O8CMU,Unknown,2023-12-07,EcoSecurities,"{""latitude"":-14.235004,""longitude"":-51.92528}",Brazil NovaGerar Landfill Gas to Energy Project,NA
3,HG20LFVJPFURJJ4M6E6MS07HTOZTR4,La Esperanza Hydroelectric Project,PA,Energy industries (renewable - / non-renewable...,Hydro,Registered,CDM - AMS-I.D.,Det Norske Veritas- CUK,2004-07-04 00:00:00,Central America,...,CDM Registry,tCO2e,https://cdm.unfccc.int,HG20LFVJPFURJJ4M6E6MS07HTOZTR4,Unknown,2023-12-07,"WB-CF, 2E Carbon Access","{""latitude"":15.199999,""longitude"":-86.241905}",La Esperanza Hydroelectric Project,NA
4,V3ZBEVSXGFNT31E2CECR2NYE93FW5L,Project for GHG Emission Reduction by Thermal ...,PA,Fugitive emissions from production and consump...,HFCs,Registered,CDM - AM0001,Japan Quality Assurance Organisation (JQA),2005-12-06 00:00:00,Eastern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,V3ZBEVSXGFNT31E2CECR2NYE93FW5L,Unknown,2023-12-07,"WB-CF, SEPA FECO","{""latitude"":35.86166,""longitude"":104.195397}",Project for GHG Emission Reduction by Thermal ...,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13168,QJD9XJ5HBYCRZR6YV9ELX4WLEC5BJ1,Reduction in steam consumption through revampi...,PA,Energy demand,EE Industry,Withdrawn,CDM - AM0018,Bureau Veritas Certification Holding SAS (BVCH),2007-03-21 00:00:00,Southern Asia,...,CDM Registry,tCO2e,https://cdm.unfccc.int,QJD9XJ5HBYCRZR6YV9ELX4WLEC5BJ1,Unknown,2023-12-07,Rashtriya Chemicals & Fertilizers,"{""latitude"":20.593684,""longitude"":78.96288}",Reduction in steam consumption through revampi...,NA
13169,4FFA579VM5UA6G3BHJYNRH5Q5Q6S6W,Usina Petribu Renewable Generation with Sugarc...,PA,Energy industries (renewable - / non-renewable...,Biomass Energy,Withdrawn,CDM - ACM0006,SGS United Kingdom Limited (SGS),2006-04-12 00:00:00,South America,...,CDM Registry,tCO2e,https://cdm.unfccc.int,4FFA579VM5UA6G3BHJYNRH5Q5Q6S6W,Unknown,2023-12-07,MaxAmbiental,"{""latitude"":-14.235004,""longitude"":-51.92528}",Usina Petribu Renewable Generation with Sugarc...,NA
13170,R33WIRKXKCNOCGHYW84XRF3SJ0KKNI,Power generation from renewable sources  Aiur...,PA,Energy industries (renewable - / non-renewable...,Hydro,Withdrawn,CDM - ACM0002,RINA Services S.p.A. (RINA),2009-05-08 00:00:00,South America,...,CDM Registry,tCO2e,https://cdm.unfccc.int,R33WIRKXKCNOCGHYW84XRF3SJ0KKNI,Unknown,2023-12-07,Waycarbon,"{""latitude"":-14.235004,""longitude"":-51.92528}",Power generation from renewable sources  Aiur...,NA
13171,GT5ME4TC32NKLA6A22ZBR00FYBC54N,Roaring 40s Wind Farms Private Limited.,PA,Energy industries (renewable - / non-renewable...,Wind,Validated,CDM - ACM0002,Det Norske Veritas- CUK,2007-0

In [48]:

df_issuances = pd.read_excel("./input/CDM-Issuances-November.xlsx")
# df_issuances.drop('DOE', axis=1, inplace=True)
df_issuances.drop('PA/PoA', axis=1, inplace=True)
df_issuances.drop('CP', axis=1, inplace=True)
df_issuances.drop('Monitoring report number', axis=1, inplace=True)
df_issuances.drop('Last_updated', axis=1, inplace=True)
df_issuances.drop('Project type (UNEP DTU)', axis=1, inplace=True)
df_issuances.drop('Project subtype (UNEP DTU)', axis=1, inplace=True)
# df_issuances.drop('Units - Total', axis=1, inplace=True)
df_issuances.drop('Issuance date', axis=1, inplace=True)
df_issuances.drop('HostParty', axis=1, inplace=True)


issuances_columns={
    "CDM project reference number":"projectId",
    "DOE":"verificationBody",
    "Issuance process ID":"id",
    "Monitoring report started":"startDate",
    "Monitoring report ended":"endDate",
     "Units - Total":"unitCount",
}
df_issuances['Monitoring report started'] = pd.to_datetime(df_issuances['Monitoring report started'])
df_issuances['Monitoring report ended'] = pd.to_datetime(df_issuances['Monitoring report ended'])
df_issuances['CDM project reference number'] = df_issuances['CDM project reference number'].map(id_to_id)
df_issuances = df_issuances[issuances_columns.keys()].rename(columns=issuances_columns)


df_issuances['verificationReportDate']=df_issuances['projectId'].map(verification_report_date_map)
df_issuances['verificationApproach']="NA"



df_issuances = fillna_by_dtype(df_issuances)
print(len(df_issuances['projectId'].unique()))
df_issuances


# df_aggregated = df_issuances.groupby('projectId')['unitCount'].sum().reset_index()
df_iss_completed = pd.read_excel("./input/CDM_Issuance_Completed_November.xlsx")
df_iss_completed['Project number'] =df_iss_completed['Project number'].map(id_to_id)

df_iss_completed

# df_aggregated2 = df_iss_completed.groupby('Project number')['Total'].sum().reset_index()
# df_aggregated2.rename(columns={'Project number': 'projectId'}, inplace=True)
# df_aggregated2.rename(columns={'Total': 'unitCount'}, inplace=True)
def rename_duplicates(df, column_name):
   
    df_copy = df.copy()
    duplic = df_copy.duplicated(subset=column_name, keep=False)
    duplicates = df_copy[duplic].copy()
    duplicates['dup_count'] = duplicates.groupby(column_name).cumcount() + 1
    duplicates[column_name] = duplicates[column_name].astype(str) + '-' + duplicates['dup_count'].astype(str)
    duplicates.drop('dup_count', axis=1, inplace=True)
    df_copy.update(duplicates)
    return df_copy

print(len(df_iss_completed))
df_issuances = rename_duplicates(df_issuances, 'id')

df_issuances= df_issuances.merge(
    df_iss_completed, 
    left_on=['projectId', 'unitCount'], 
    right_on=['Project number', 'Total'],
    how='left'
)

df_issuances.drop('Completion date', axis=1, inplace=True)
df_issuances.drop('Transaction type', axis=1, inplace=True)
df_issuances.drop('Total', axis=1, inplace=True)
df_issuances.drop('Project number', axis=1, inplace=True)
df_issuances.drop('PARTY CODE', axis=1, inplace=True)
df_issuances.drop('CP', axis=1, inplace=True)
df_issuances.drop('Unit Type', axis=1, inplace=True)
df_issuances["Status"] = "Held"
df_issuances.drop_duplicates(subset='id', keep='first', inplace=True)
# df_dup_check = df_issuances[df_issuances.duplicated(subset='id', keep=False)]

# df_dup_check
# df_issuances.to_excel("propercleanissuance.xlsx")


nan_exists = df_issuances['SERIAL RANGE'].isna().any()
nan_exists

3703
12570


False

In [50]:
df_units = pd.read_excel("./input/CDM-Issuances-November.xlsx")


units_columns={
    "HostParty":"unitOwner",
    "CDM project reference number":"projectId",
    "Issuance process ID":"issuanceId",
    "DOE":"verificationBody",
    "Issuance process ID":"id",
   "Units - Total":"unitCount",
   "Issuance date":"vintageYear",

}

df_units = df_units[units_columns.keys()].rename(columns=units_columns)
df_units["unitOwner"] = df_units["unitOwner"].map(country_name_map)
df_units["projectId"] = df_units["projectId"].map(id_to_id)
df_units["countryJurisdictionOfOwner"] = df_units["unitOwner"]
df_units["inCountryJurisdictionOfOwner"] = ""
df_units["projectLocationId"] = ""
df_units["geographyIdentifier"] = df_units["unitOwner"].map(geography_map)
df_units["correspondingAdjustmentDeclaration"] = "Unknown"
df_units["unitRegistryLink"] = "http://cdm.unfccc.int/"
df_units["vintageYear"] = df_units["vintageYear"].astype(str).str[:4]
df_units["unitStatus"] = "Held"
df_units["unitType"] = ""
df_units["unitBlockStart"] = ""
df_units["unitBlockEnd"] = ""


df_units = fillna_by_dtype(df_units)

df_units

,unitOwner,projectId,id,verificationBody,unitCount,vintageYear,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,projectLocationId,geographyIdentifier,correspondingAdjustmentDeclaration,unitRegistryLink,unitStatus,unitType,unitBlockStart,unitBlockEnd
0,India,IBK3RIR7HVE1SZLK7YZMZPZ02UW7TS,4KEarth1568615888.93,KEarth,20519,2020,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
1,India,KZPP9BVTCRTYEUSRYB2L0ZNRJUDF2X,4KEarth1568616446.5,KEarth,18925,2020,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
2,India,P2UF559LX39J2VDPTZ13PAZUO3DRVV,4KEarth1585297661.77,KEarth,24595,2021,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
3,Thailand,AXF2WJ2NOM4CHZGDOJW9S9DINIAQWS,4KEarth1596186178.27,KEarth,43219,2021,Thailand,,,"{""latitude"":15.870032,""longitude"":100.992541}",Unknown,http://cdm.unfccc.int/,Held,,,
4,India,JOUI5BA4DLQS9MVXDKRLL1HHFVQ4M3,4KEarth1599113457.49,KEarth,116637,2022,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12543,India,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828753.52,No info,1231148,2022,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
12544,India,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828921.37,No info,1128556,2022,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,
12545,China,D4CRFLKI60AFK14YEHRDNPY16829DF,sergeyf731650363867.4,No info,434350,2022,China,,,"{""latitude"":35.86166,""longitude"":104.195397}",Unknown,http://cdm.unfccc.int/,Held,,,
12546,India,BN70H0KER9GB6Y0BPJBJ3Q4IBIXK7C,slenzen1552989570.98,No info,102514,2020,India,,,"{""latitude"":20.593684,""longitude"":78.96288}",Unknown,http://cdm.unfccc.int/,Held,,,


In [51]:
df_retired = pd.read_excel("./input/CDM_Retired_November.xlsx")
df_retired['PARTY CODE'] = df_retired['Project Identifier'].str.split('-').str[0]
df_retired['Project Identifier'] = df_retired['Project Identifier'].str.split('-').str[1].astype(int)



vintage_year_map= df_units.set_index('projectId')['vintageYear'].to_dict()
issuance_id_map= df_units.set_index('projectId')['id'].to_dict()
verif_body_units_map= df_units.set_index('projectId')['verificationBody'].to_dict()
geog_identifier_units_map = df_units.set_index('projectId')['geographyIdentifier'].to_dict()




# df_retired


retired_columns={
    "Status":"unitStatus",
    "Total":"unitTotal",
    "PARTY CODE":"unitOwner",
    "Project Identifier":"projectId",
    "Serial Range":"Serial Range",
    "Transaction Type":"unitStatus"
}



df_retired = df_retired[retired_columns.keys()].rename(columns=retired_columns)
df_retired[['unitBlockStart', 'unitBlockEnd']] = df_retired['Serial Range'].str.split('-', 1, expand=True)
df_retired['unitBlockStart'] = df_retired['unitBlockStart'].fillna(0).astype(int)
df_retired['unitBlockEnd'] = df_retired['unitBlockEnd'].fillna(0).astype(int)
df_retired.drop('Serial Range', axis=1, inplace=True)
df_retired['projectId']=df_retired['projectId'].map(id_to_id)

df_retired["unitOwner"]= df_retired['unitOwner'].map(country_name_map)
df_retired["countryJurisdictionOfOwner"] = df_retired["unitOwner"]
df_retired['inCountryJurisdictionOfOwner'] = ""


df_retired["unitType"] = ""
df_retired["correspondingAdjustmentDeclaration"] = "Unknown"
df_retired["unitRegistryLink"] = "http://cdm.unfccc.int/"


df_needed = df_issuances[['projectId', 'id', 'startDate', 'endDate', 
           'unitCount', 'SERIAL RANGE']]
df_needed[['start', 'end']] = df_needed['SERIAL RANGE'].str.split(' - ', expand=True)

# Convert 'start' and 'end' to integers
df_needed['start'] = df_needed['start'].astype(int)
df_needed['end'] = df_needed['end'].astype(int)
df_needed['yearDiff'] = df_needed['endDate'].dt.year - df_needed['startDate'].dt.year
df_needed['countDiff'] = df_needed['end']-df_needed['start']




# Todo df_retired["vintageYear"]= df_retired['projectId'].map(vintage_year_map)
# Todo df_retired["id"]= df_retired['projectId'].map(issuance_id_map)
df_retired["verificationBody"]= df_retired['projectId'].map(verif_body_units_map)
df_retired['geographyIdentifier']= df_retired['projectId'].map(geog_identifier_units_map)
# Todo df_retired["projectLocationId"] = ""
df_retired = fillna_by_dtype(df_retired)
df_retired["unitStatus"] = "Retired"

# Only 1556 issuance
df_filtered = df_needed[df_needed['startDate'].dt.year >= 2016]

df_filtered
len(df_filtered['id'].unique())



/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_2353/1555901657.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_needed[['start', 'end']] = df_needed['SERIAL RANGE'].str.split(' - ', expand=True)
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_2353/1555901657.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_needed[['start', 'end']] = df_needed['SERIAL RANGE'].str.split(' - ', expand=True)
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_2353/1555901657.py:51: Se

1551

In [52]:
df_retired

,unitStatus,unitTotal,unitOwner,projectId,unitStatus,unitBlockStart,unitBlockEnd,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,unitType,correspondingAdjustmentDeclaration,unitRegistryLink,verificationBody,geographyIdentifier
0,Retired,5294,Brazil,DWXTGTQLAORUROS9KPVZJUSGI8UK70,Retired,65142364,65147657,Brazil,,,Unknown,http://cdm.unfccc.int/,RINA,"{""latitude"":-14.235004,""longitude"":-51.92528}"
1,Retired,5294,Brazil,8J911GX7NLREEIL5L09MKJTI9USERE,Retired,60482913,60488206,Brazil,,,Unknown,http://cdm.unfccc.int/,TUV NORD,"{""latitude"":-14.235004,""longitude"":-51.92528}"
2,Retired,5000,Brazil,O5C8ZV4YZTL3001AX5RD5V1OL5OM0A,Retired,57297956,57302955,Brazil,,,Unknown,http://cdm.unfccc.int/,DNV,"{""latitude"":-14.235004,""longitude"":-51.92528}"
3,Retired,5000,Brazil,HY2WB5CJ5DZTLBFLQE55ZHT0XK2DO2,Retired,61682096,61687095,Brazil,,,Unknown,http://cdm.unfccc.int/,TUV SUD,"{""latitude"":-14.235004,""longitude"":-51.92528}"
4,Retired,5000,Brazil,0IET49KTMORLOP6NX7Q56DF7HXTMPV,Retired,57923146,57928145,Brazil,,,Unknown,http://cdm.unfccc.int/,DNV,"{""latitude"":-14.235004,""longitude"":-51.92528}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27741,Retired,3,Chile,L9H0347KVT9F7MAMD49TAYUJLMFHEW,Retired,41951955,41951957,Chile,,,Unknown,http://cdm.unfccc.int/,TUV NORD,"{""latitude"":-35.675147,""longitude"":-71.542969}"
27742,Retired,1,China,B90WLUX8QKF2591LN21LEVJ82HOO75,Retired,1185996159,1185996159,China,,,Unknown,http://cdm.unfccc.int/,LGAI,"{""latitude"":35.86166,""longitude"":104.195397}"
27743,Retired,186,Malawi,YHNSTOHUC3NLV2APHHB6SCWA61Q46T,Retired,207935,208120,Malawi,,,Unknown,http://cdm.unfccc.int/,TUV SUD,"{""latitude"":-13.254308,""longitude"":34.301525}"
27744,Retired,129,India,82ORS4DFFAE3F7JI64VMOSYQEYXGCB,Retired,279636945,279637073,India,,,Unknown,http://cdm.unfccc.int/,LGAI,"{""latitude"":20.593684,""longitude"":78.96288}"


In [53]:
df_merged = pd.merge(df_retired, df_needed, on='projectId', how='left', suffixes=('_retired', '_issuance'))

# Filter rows where unitBlockStart and unitBlockEnd are within the start and end range
df_linked = df_merged[(df_merged['unitBlockStart'] >= df_merged['start']) & (df_merged['unitBlockEnd'] <= df_merged['end'])]





def checker_playaround(projectId):

    filtered_braz = df_linked[df_linked['projectId'] == projectId]


    fil_ori = df_retired[df_retired['projectId']== projectId]

    fil_haha = df_needed[df_needed['projectId']== projectId]

    filtered_braz.to_excel("testlook.xlsx")
    print("unique issuance id here is",filtered_braz['id'].unique())
    print("unique issuance id here is",fil_haha['id'].unique())
    print("total length 1",len(fil_ori))
    print("total length 2",len(filtered_braz))




df_linked

,unitStatus,unitTotal,unitOwner,projectId,unitStatus,unitBlockStart,unitBlockEnd,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,unitType,...,geographyIdentifier,id,startDate,endDate,unitCount,SERIAL RANGE,start,end,yearDiff,countDiff
0,Retired,5294,Brazil,DWXTGTQLAORUROS9KPVZJUSGI8UK70,Retired,65142364,65147657,Brazil,,,...,"{""latitude"":-14.235004,""longitude"":-51.92528}",DNV-CUK1293110659.42,2000-11-10,2010-11-09,4072355.0,63069791 - 67142145,6.306979e+07,6.714214e+07,10.0,4072354.0
6,Retired,5294,Brazil,8J911GX7NLREEIL5L09MKJTI9USERE,Retired,60482913,60488206,Brazil,,,...,"{""latitude"":-14.235004,""longitude"":-51.92528}",DNV-CUK1299845791.9,2010-07-01,2011-02-28,157369.0,60445682 - 60603050,6.044568e+07,6.060305e+07,1.0,157368.0
18,Retired,5000,Brazil,O5C8ZV4YZTL3001AX5RD5V1OL5OM0A,Retired,57297956,57302955,Brazil,,,...,"{""latitude"":-14.235004,""longitude"":-51.92528}",DNV-CUK1264416604.37,2006-10-01,2009-09-30,89641.0,57287379 - 57377019,5.728738e+07,5.737702e+07,3.0,89640.0
36,Retired,5000,Brazil,HY2WB5CJ5DZTLBFLQE55ZHT0XK2DO2,Retired,61682096,61687095,Brazil,,,...,"{""latitude"":-14.235004,""longitude"":-51.92528}",TUEV-SUED1317711892.85,2011-06-01,2011-09-30,93646.0,61680223 - 61773868,6.168022e+07,6.177387e+07,0.0,93645.0
43,Retired,5000,Brazil,0IET49KTMORLOP6NX7Q56DF7HXTMPV,Retired,57923146,57928145,Brazil,,,...,"{""latitude"":-14.235004,""longitude"":-51.92528}",DNV-CUK1265191517.98,2008-06-01,2009-05-31,157914.0,57831988 - 57989901,5.783199e+07,5.798990e+07,1.0,157913.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130472,Retired,3,Chile,L9H0347KVT9F7MAMD49TAYUJLMFHEW,Retired,41951955,41951957,Chile,,,...,"{""latitude"":-35.675147,""longitude"":-71.542969}",Applus1654182155.66,2013-07-30,2019-03-17,1541464.0,41762547 - 43304010,4.176255e+07,4.330401e+07,6.0,1541463.0
130476,Retired,1,China,B90WLUX8QKF2591LN21LEVJ82HOO75,Retired,1185996159,1185996159,China,,,...,"{""latitude"":35.86166,""longitude"":104.195397}",Applus1654080342.92-2,2013-01-01,2017-12-04,707684.0,1185849127 - 1186556810,1.185849e+09,1.186557e+09,4.0,707683.0
130478,Retired,186,Malawi,YHNSTOHUC3NLV2APHHB6SCWA61Q46T,Retired,207935,208120,Malawi,,,...,"{""latitude"":-13.254308,""longitude"":34.301525}",TUEV-SUED1446721135.84,2014-10-27,2015-08-05,35318.0,172850 - 208167,1.728500e+05,2.081670e+05,1.0,35317.0
130479,Retired,129,India,82ORS4DFFAE3F7JI64VMOSYQEYXGCB,Retired,279636945,279637073,India,,,...,"{""latitude"":20.593684,""longitude"":78.96288}",Applus1557750802.84-1,2013-01-01,2016-10-01,71478.0,279587171 - 279658648,2.795872e+08,2.796586e+08,3.0,71477.0


In [54]:
df_linked['startDate'] = pd.to_datetime(df_linked['startDate'])
df_linked['endDate'] = pd.to_datetime(df_linked['endDate'])

# Function to calculate vintage year
def calculate_vintage_year(row):
    total_years = row['endDate'].year - row['startDate'].year
    total_years = max(total_years, 1)  # Ensure total_years is at least 1
    units_per_year = row['unitCount'] / total_years
    year_offset = (row['unitBlockStart'] - row['start']) // units_per_year
    return row['startDate'].year + int(year_offset)

# Apply the function to each row
df_linked['vintageYear'] = df_linked.apply(calculate_vintage_year, axis=1)




checker_playaround("B90WLUX8QKF2591LN21LEVJ82HOO75")

df_linked_2016 = df_linked[df_linked['startDate'].dt.year >= 2016]


df_linked.columns

# total 3392 retired units
df_linked_2016

print("Issuances belong to retired units",len(df_linked_2016['id'].unique()))



/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_2353/911841154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_linked['startDate'] = pd.to_datetime(df_linked['startDate'])
/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_2353/911841154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_linked['endDate'] = pd.to_datetime(df_linked['endDate'])


unique issuance id here is ['Applus1654080342.92-2' 'Applus1654080342.92-1']
unique issuance id here is ['Applus1654080342.92-1' 'Applus1654080342.92-2']
total length 1 66
total length 2 66
Issuances belong to retired units 450


/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_2353/911841154.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_linked['vintageYear'] = df_linked.apply(calculate_vintage_year, axis=1)


In [55]:
df_retired_clean = df_linked.copy()


print(df_retired_clean.columns)



columns_df_linked = ['unitStatus', 'unitTotal', 'unitOwner', 'projectId', 'unitStatus',
                     'unitBlockStart', 'unitBlockEnd', 'countryJurisdictionOfOwner',
                     'inCountryJurisdictionOfOwner', 'unitType',
                     'correspondingAdjustmentDeclaration', 'unitRegistryLink',
                     'verificationBody', 'geographyIdentifier', 'id', 'startDate', 'endDate',
                     'unitCount', 'SERIAL RANGE', 'start', 'end', 'yearDiff', 'countDiff',
                     'vintageYear']

# List of columns in df_retired_need
columns_df_retired_need = ['unitStatus', 'unitTotal', 'unitOwner', 'projectId', 'unitStatus',
                           'unitBlockStart', 'unitBlockEnd', 'countryJurisdictionOfOwner',
                           'inCountryJurisdictionOfOwner', 'unitType',
                           'correspondingAdjustmentDeclaration', 'unitRegistryLink',
                           'verificationBody', 'geographyIdentifier', 'vintageYear']

# Columns in df_retired_need but not in df_linked
columns_not_in_linked = set(columns_df_retired_need) - set(columns_df_linked)

# Print the result
print("not in the linke dis", columns_not_in_linked)

# WILL NEED ID LATER
df_retired_clean = df_retired_clean[['unitStatus', 'unitTotal', 'unitOwner', 'projectId', 'unitStatus',
       'unitBlockStart', 'unitBlockEnd', 'countryJurisdictionOfOwner',
       'inCountryJurisdictionOfOwner', 'unitType',
       'correspondingAdjustmentDeclaration', 'unitRegistryLink',
       'verificationBody', 'geographyIdentifier','vintageYear','id']]

df_retired_clean.rename(columns={'unitTotal': 'unitCount'}, inplace=True)

df_retired_clean = df_retired_clean.loc[:,~df_retired_clean.columns.duplicated()]
df_retired_clean



Index(['unitStatus', 'unitTotal', 'unitOwner', 'projectId', 'unitStatus',
       'unitBlockStart', 'unitBlockEnd', 'countryJurisdictionOfOwner',
       'inCountryJurisdictionOfOwner', 'unitType',
       'correspondingAdjustmentDeclaration', 'unitRegistryLink',
       'verificationBody', 'geographyIdentifier', 'id', 'startDate', 'endDate',
       'unitCount', 'SERIAL RANGE', 'start', 'end', 'yearDiff', 'countDiff',
       'vintageYear'],
      dtype='object')
not in the linke dis set()


,unitStatus,unitCount,unitOwner,projectId,unitBlockStart,unitBlockEnd,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,unitType,correspondingAdjustmentDeclaration,unitRegistryLink,verificationBody,geographyIdentifier,vintageYear,id
0,Retired,5294,Brazil,DWXTGTQLAORUROS9KPVZJUSGI8UK70,65142364,65147657,Brazil,,,Unknown,http://cdm.unfccc.int/,RINA,"{""latitude"":-14.235004,""longitude"":-51.92528}",2005,DNV-CUK1293110659.42
6,Retired,5294,Brazil,8J911GX7NLREEIL5L09MKJTI9USERE,60482913,60488206,Brazil,,,Unknown,http://cdm.unfccc.int/,TUV NORD,"{""latitude"":-14.235004,""longitude"":-51.92528}",2010,DNV-CUK1299845791.9
18,Retired,5000,Brazil,O5C8ZV4YZTL3001AX5RD5V1OL5OM0A,57297956,57302955,Brazil,,,Unknown,http://cdm.unfccc.int/,DNV,"{""latitude"":-14.235004,""longitude"":-51.92528}",2006,DNV-CUK1264416604.37
36,Retired,5000,Brazil,HY2WB5CJ5DZTLBFLQE55ZHT0XK2DO2,61682096,61687095,Brazil,,,Unknown,http://cdm.unfccc.int/,TUV SUD,"{""latitude"":-14.235004,""longitude"":-51.92528}",2011,TUEV-SUED1317711892.85
43,Retired,5000,Brazil,0IET49KTMORLOP6NX7Q56DF7HXTMPV,57923146,57928145,Brazil,,,Unknown,http://cdm.unfccc.int/,DNV,"{""latitude"":-14.235004,""longitude"":-51.92528}",2008,DNV-CUK1265191517.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130472,Retired,3,Chile,L9H0347KVT9F7MAMD49TAYUJLMFHEW,41951955,41951957,Chile,,,Unknown,http://cdm.unfccc.int/,TUV NORD,"{""latitude"":-35.675147,""longitude"":-71.542969}",2013,Applus1654182155.66
130476,Retired,1,China,B90WLUX8QKF2591LN21LEVJ82HOO75,1185996159,1185996159,China,,,Unknown,http://cdm.unfccc.int/,LGAI,"{""latitude"":35.86166,""longitude"":104.195397}",2013,Applus1654080342.92-2
130478,Retired,186,Malawi,YHNSTOHUC3NLV2APHHB6SCWA61Q46T,207935,208120,Malawi,,,Unknown,http://cdm.unfccc.int/,TUV SUD,"{""latitude"":-13.254308,""longitude"":34.301525}",2014,TUEV-SUED1446721135.84
130479,Retired,129,India,82ORS4DFFAE3F7JI64VMOSYQEYXGCB,279636945,279637073,India,,,Unknown,http://cdm.unfccc.int/,LGAI,"{""latitude"":20.593684,""longitude"":78.96288}",2015,Applus1557750802.84-1


In [67]:
df_units_held = df_linked.copy()

print("lenoriginal dfUnitsHeld",len(df_units_held))

grouped_total = df_units_held.groupby('id')['unitTotal'].sum().reset_index()
grouped_total.rename(columns={'unitTotal': 'aggregatedUnitTotal'}, inplace=True)

# Step 2: Merge the aggregated data back into df_linked
df_units_held = df_units_held.merge(grouped_total, on='id', how='left')

# Step 3: Calculate totalHeld
df_units_held['totalHeld'] = df_units_held['unitCount'] - df_units_held['aggregatedUnitTotal']


# ['unitStatus', 'unitTotal', 'unitOwner', 'projectId', 'unitStatus', 
#            'unitBlockStart', 'unitBlockEnd', 'countryJurisdictionOfOwner', 
#            'inCountryJurisdictionOfOwner', 'unitType', 'endDate', 'unitCount', 
#            'SERIAL RANGE', 'start', 'end', 'yearDiff', 'countDiff', 
#            'vintageYear', 'aggregatedUnitTotal', 'totalHeld']


totalHeld_mapping = df_units_held.set_index('id')['totalHeld'].to_dict()
projectId_country_map = df_units.set_index('projectId')['unitOwner'].to_dict()



df_units_held = df_issuances.copy()

df_units_held['totalHeld'] = df_issuances['id'].map(totalHeld_mapping)
df_units_held['totalHeld'] = df_units_held['totalHeld'].fillna(df_units_held['unitCount'])
df_see = df_units_held[['id','totalHeld','unitCount']]
df_units_held.drop('unitCount',axis=1,inplace=True)

print("df_units_held column is",df_units_held.columns)



df_units_held.rename(columns={'totalHeld': 'unitCount'}, inplace=True)


# df_units_held.to_excel("df_heldinspect.xlsx")

df_units_held[['unitBlockStart', 'unitBlockEnd']] = df_units_held['SERIAL RANGE'].str.split('-', 1, expand=True)
df_units_held.drop('SERIAL RANGE', axis=1, inplace=True)

df_units_held['unitOwner'] = df_units_held['projectId'].map(projectId_country_map)

df_units_held["countryJurisdictionOfOwner"] = df_units_held["unitOwner"]
df_units_held["inCountryJurisdictionOfOwner"] = ""
df_units_held["projectLocationId"] = ""


df_units_held.rename(columns={'Project number': 'projectId'}, inplace=True)
df_units_held.rename(columns={'Total': 'unitCount'}, inplace=True)
# df_units_held["vintageYear"] = df_units_held["startDate"].astype(str).str[:4]
df_units_held["geographyIdentifier"] = df_units_held["unitOwner"].map(geography_map)
df_units_held.rename(columns={'Status': 'unitStatus'}, inplace=True)
df_units_held['unitRegistryLink'] = "http://cdm.unfccc.int/"
df_units_held['correspondingAdjustmentDeclaration'] = "Unknown"
df_units_held['unitType'] =""
columns_to_drop = ['verificationReportDate', 'startDate', 'projectLocationId', 
                   'endDate', 'verificationApproach']

# Dropping the columns
df_units_held = df_units_held.drop(columns=columns_to_drop)

df_units_held.columns


print(set(df_units_held.columns)-set(df_retired_clean.columns))
print(set(df_retired_clean.columns)-set(df_units_held.columns))


print(df_retired_clean.columns)

print(df_units_held.columns)

print(df_see['totalHeld'].unique())
# df_units_held

lenoriginal dfUnitsHeld 27749
df_units_held column is Index(['projectId', 'verificationBody', 'id', 'startDate', 'endDate',
       'verificationReportDate', 'verificationApproach', 'Status',
       'SERIAL RANGE', 'totalHeld'],
      dtype='object')
set()
{'vintageYear'}
Index(['unitStatus', 'unitCount', 'unitOwner', 'projectId', 'unitBlockStart',
       'unitBlockEnd', 'countryJurisdictionOfOwner',
       'inCountryJurisdictionOfOwner', 'unitType',
       'correspondingAdjustmentDeclaration', 'unitRegistryLink',
       'verificationBody', 'geographyIdentifier', 'vintageYear', 'id'],
      dtype='object')
Index(['projectId', 'verificationBody', 'id', 'unitStatus', 'unitCount',
       'unitBlockStart', 'unitBlockEnd', 'unitOwner',
       'countryJurisdictionOfOwner', 'inCountryJurisdictionOfOwner',
       'geographyIdentifier', 'unitRegistryLink',
       'correspondingAdjustmentDeclaration', 'unitType'],
      dtype='object')
[  410.   378. 24595. ... 37420. 98346. 51790.]


In [40]:
# CHecking qc
df_units_held.to_excel("./output/CDM_Held_Units_Cleaned.xlsx")
df_retired_clean.to_excel("./output/CDM_Retired_Units_Cleaned.xlsx")
# df_issuances.to_excel("./output/CDM_Issuances_Cleaned.xlsx")

df_issuances_cleaned = df_issuances[["verificationBody","projectId","id","startDate","endDate","verificationReportDate","verificationApproach"]]
unique_ids_held = df_units_held['id'].unique()
unique_ids_retired = df_retired_clean['id'].unique()
unique_ids_issuances = df_issuances['id'].unique()


df_filtered = df_needed[df_needed['startDate'].dt.year >= 2016]



print(set((unique_ids_retired))-set((unique_ids_held)))
print(set((unique_ids_retired))-set((unique_ids_issuances)))
print(set((unique_ids_issuances))-set((unique_ids_held)))
print(set((unique_ids_held))-set((unique_ids_issuances)))

def convert_timestamp_to_string(x):
    if isinstance(x, pd.Timestamp):
        return x.strftime('%Y-%m-%d %H:%M:%S')
    return x

# Columns that contain Timestamps
timestamp_columns = ['startDate', 'endDate', 'verificationReportDate']

# Apply the conversion to the specific columns

# Apply the conversion to the existing timestamp columns
df_issuances_cleaned[timestamp_columns] = df_issuances_cleaned[timestamp_columns].applymap(convert_timestamp_to_string)
issuances_grouped = df_issuances_cleaned.groupby('projectId').apply(lambda x: x.to_dict(orient='records')).reset_index(name='issuances')

issuances_grouped['issuances'][0]


df_issuances_cleaned

# df_issuances_cleaned


set()
set()
set()
set()


/var/folders/5x/w7k9bsls4fj7q9x5f3xxvt5m0000gn/T/ipykernel_2353/1025292522.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_issuances_cleaned[timestamp_columns] = df_issuances_cleaned[timestamp_columns].applymap(convert_timestamp_to_string)


,verificationBody,projectId,id,startDate,endDate,verificationReportDate,verificationApproach
0,KEarth,IBK3RIR7HVE1SZLK7YZMZPZ02UW7TS,4KEarth1568615888.93,2017-04-01 00:00:00,2019-07-31 00:00:00,2014-01-01 00:00:00,NA
1,KEarth,KZPP9BVTCRTYEUSRYB2L0ZNRJUDF2X,4KEarth1568616446.5,2017-04-01 00:00:00,2019-07-31 00:00:00,2014-05-31 00:00:00,NA
2,KEarth,P2UF559LX39J2VDPTZ13PAZUO3DRVV,4KEarth1585297661.77,2018-01-01 00:00:00,2019-12-31 00:00:00,2011-01-01 00:00:00,NA
3,KEarth,AXF2WJ2NOM4CHZGDOJW9S9DINIAQWS,4KEarth1596186178.27,2018-01-01 00:00:00,2020-03-21 00:00:00,2013-03-22 00:00:00,NA
4,KEarth,JOUI5BA4DLQS9MVXDKRLL1HHFVQ4M3,4KEarth1599113457.49,2015-04-01 00:00:00,2017-12-31 00:00:00,2013-10-10 00:00:00,NA
...,...,...,...,...,...,...,...
12551,No info,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828753.52,2016-09-01 00:00:00,2017-09-30 00:00:00,2013-03-15 00:00:00,NA
12552,No info,3A5UYHPO8SISCX8T4BJ1AMN6SO780J,sergeyf731618828921.37,2017-10-01 00:00:00,2018-12-08 00:00:00,2013-03-15 00:00:00,NA
12553,No info,D4CRFLKI60AFK14YEHRDNPY16829DF,sergeyf731650363867.4,2016-10-01 00:00:00,2017-06-30 00:00:00,2012-11-01 00:00:00,NA
12554,No info,BN70H0KER9GB6Y0BPJBJ3Q4IBIXK7C,slenzen1552989570.98,2017-01-01 00:00:00,2019-01-31 00:00:00,2013-12-27 00:00:00,NA


In [57]:

api_endpoint = "http://localhost:31310/v1/projects"

def datetime_to_string(value):
    if isinstance(value, datetime):
        return value.strftime("%Y-%m-%d")
    return value
# Function to send POST request for each row in the DataFrame
def post_projects(df):
    # Iterate over each row in the DataFrame
    project_to_uuid_mapping = {}
    i=0
    for index, row in df[0:5000].iterrows():
        # Construct the request body from the row, excluding 'projectLocation'
        # and adding 'projectLocations' which is not in df_project
        request_body = row.to_dict()
        request_body = {k: datetime_to_string(v) for k, v in row.to_dict().items()}
        # Example additional data for 'projectLocations' as it is not present in the DataFrame
        request_body['projectLocations'] = [
            {
                "country": request_body.get('country', 'Unknown'),
                "inCountryRegion": request_body.get('inCountryRegion', 'Unknown'),
                "geographicIdentifier":request_body.get('geographicIdentifier',{})  # replace with actual geographicIdentifier if available
            }
        ]

        issuance_ids = []
        issuances_for_api = []
        if row.get('issuances') is not None and isinstance(row.get('issuances'), list):
            for issuance in row.get('issuances', []):
                # Store the 'id' for later use
                issuance_id = issuance.get('id')
                issuance_ids.append(issuance_id)
                
                # Remove 'id' from the issuance data sent to the API
                issuance_for_api = {key: value for key, value in issuance.items() if key not in ['id', 'projectId']}
                issuances_for_api.append(issuance_for_api)
            request_body['issuances'] = issuances_for_api
        else:

            request_body.pop('issuances', None)

        
        # Remove the keys that are not needed for the POST request
        request_body.pop('country', None)
        request_body.pop('inCountryRegion', None)
        request_body.pop('geographicIdentifier', None)

       
        # Send POST request
        try:
            response = requests.post(api_endpoint, json=request_body)
            # Check if the request was successful
            if response.status_code == 200:
               response_data = response.json()
               print(f"Successfully posted project with ID: {row['projectId']}, UUID: {response_data.get('uuid')}")
             
            else:
                print("i is",i)
                print(request_body)
                print(f"Failed to post project with ID: {row['originProjectId']}. Status code: {response.status_code}. Message: {response.json()}")
                break
        except RequestException as e:
            print(request_body)
            print(f"Request failed for project with ID: {row['originProjectId']}. Error: {e}")
            break

        i+=1
    return project_to_uuid_mapping
# Call the function to start posting data


merged_project_issuances = pd.merge(df_cleaned_project, issuances_grouped, on='projectId', how='left')

# df_project


def get_projects_staging_uuid():
    url = "http://localhost:31310/v1/staging"

    # Make a GET request
    response = requests.get(url)

    
    # Check the response status code
    if response.status_code == 200:
        # The request was successful, and you can access the response content
        projects = response.json()
        project_id_to_uuid = {}
        project_id_to_issuances={}
        project_id_to_issuances={}
        for project in projects:

            uuid = project['uuid']
            # print(uuid)
            projectId = project['diff']['change'][0]['projectId']
            project_id_to_uuid[projectId] = uuid
            issuances = project['diff']['change'][0].get('issuances', None)

            if issuances:
                project_id_to_issuances[projectId] = issuances

        # print(uuid_to_project_id)
        return project_id_to_uuid, project_id_to_issuances
    else:
        # The request failed for some reason
        print(f"Request failed with status code {response.status_code}")



project_to_uuid_map,project_to_issuances = get_projects_staging_uuid()

print(len(project_to_uuid_map))

# fail in project num 6142

# check if there is a same project but several uuid

# print(uuid_to_project_id_map)


# merged_project_issuances.to_excel("./output/ProjectsIssuancesCombined.xlsx")


Request failed with status code 400


TypeError: cannot unpack non-iterable NoneType object

In [355]:

post_projects(merged_project_issuances)

Successfully posted project with ID: 9CPDZSI5SBOU5BRBXV7BU1JIR4EHEX, UUID: 45f07bf0-2f4a-4b4c-bf1f-33ca12ea20fc
Successfully posted project with ID: U3T39WU3P6F04ISC6OGTR0098OGZ5G, UUID: f414e261-1400-420f-a563-a97aae91631f
Successfully posted project with ID: 7SL2CIZ01HQ78NGPH2HG9DG0DCK4YO, UUID: 583881bc-22ed-42df-a55d-e8249772d68b
Successfully posted project with ID: HORREMN83GI73DZK9TFGJ5YYT6HRY0, UUID: c12057c9-c0b7-4aac-8fb0-37f842cd3260
Successfully posted project with ID: 7L6UW2UJ9WL7JU2P3GMZSGYBEV0GYK, UUID: 09b7115c-1bfb-4c92-b8aa-6034baca0b8c
Successfully posted project with ID: 7XI0PL0WS1BX1675PC76QFO09DSR1D, UUID: 0d8f1b00-c6a8-485a-b0e8-44fbd1b94abe
Successfully posted project with ID: 3X4AAJ5J0YGRUR33BEQBF2DOPZ87CI, UUID: 13f3f8b5-2283-42ab-b85b-d1278c2afaa8
Successfully posted project with ID: OXEVYP45BDLBC0N23JPOLZ72NDRBMO, UUID: a9383df8-0e24-4eb9-bf0f-77e9d6163b50
Successfully posted project with ID: YB015OBLMUA1DXCFWQH8X9Z8KY88Z8, UUID: 4131f0e8-af66-4074-b856-2177b

{}

In [ ]:
#  WIll return this from staging
# {
#                             "verificationBody": "SGS",
#                             "startDate": "2003-12-31T16:00:00.000Z",
#                             "endDate": "2005-12-30T16:00:00.000Z",
#                             "verificationReportDate": "2003-12-31T16:00:00.000Z",
#                             "verificationApproach": "NA",
#                             "id": "51b6154c-edb9-44d2-bde0-cdec7de9be60",
#                             "orgUid": "0f01dc5feefbfa32d8d1352480fb96bedce530318ad32ded1b02e346b56affa4",
#                             "warehouseProjectId": "045744a2-031e-4a64-ac69-992d9fd4d9a0"
#                         },

In [31]:
project_to_issuances

df_retired_clean


merged_issuances_units = pd.merge(df_retired_clean, df_issuances_cleaned, on='id', how='left')

# merged_issuances_units
df_issuances_cleaned
df_issuances_cleaned_unique = df_issuances_cleaned.drop_duplicates(subset='id')

df_issuances_cleaned_unique

duplicates = df_issuances_cleaned.duplicated(subset='id', keep=False)

# Filter the DataFrame to only include the duplicates
df_issuances_duplicates = df_issuances_cleaned[duplicates]

# df_issuances_duplicates.to_excel("./output/duplicateIssuances.xlsx")

merged_issuances_units.drop('projectId_x', axis=1, inplace=True)

merged_issuances_units.drop('verificationBody_x', axis=1, inplace=True)
merged_issuances_units.rename(columns={'projectId_y': 'projectId'}, inplace=True)
merged_issuances_units.rename(columns={'verificationBody_y': 'verificationBody'}, inplace=True)
merged_issuances_units['correspondingAdjustmentStatus']= "Not Applicable"
merged_issuances_units['warehouseProjectId']= merged_issuances_units['projectId'].map(project_to_uuid_map)


merged_issuances_units.dropna()


NameError: name 'project_to_issuances' is not defined

In [359]:
df_checking_iss = pd.read_excel("./input/CDM-Issuances-November.xlsx")

df_checking_iss

duplic = df_checking_iss.duplicated(subset='Issuance process ID', keep=False)

# Filter the DataFrame to only include the duplicates
df_duplic = df_checking_iss[duplic]
df_duplic = df_duplic.sort_values(by='CDM project reference number', ascending=True)

df_duplic.to_excel("duplicduplic.xlsx")

len(df_checking_iss)




12548

In [ ]:
def rename_duplicates(df, column_name):
   
    df_copy = df.copy()
    duplic = df_copy.duplicated(subset=column_name, keep=False)
    duplicates = df_copy[duplic].copy()
    duplicates['dup_count'] = duplicates.groupby(column_name).cumcount() + 1
    duplicates[column_name] = duplicates[column_name].astype(str) + '-' + duplicates['dup_count'].astype(str)
    duplicates.drop('dup_count', axis=1, inplace=True)
    df_copy.update(duplicates)
    return df_copy
updated_df = rename_duplicates(df_checking_iss, 'Issuance process ID')
updated_df

